# Exploring Hacker News Posts

In this project, we'll work with a data set of submissions to a popular site [Hacker News](https://news.ycombinator.com/) 

The original data set is from [Kaggle](https://www.kaggle.com/hacker-news/hacker-news-posts), but it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling the remaining submissions.

The description of the columns in the csv file are as follows:

-  `id` - The unique identifier from Hacker News for the post
-  `title` - The title of the post
-  `url` - The URL that the post links to, if the post has a URL
-  `num_points` - The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
-  `num_comments` - The number of comments that were made on the post
-  `author` - The username of the person who submitted the post
-  `created_at` - The date and time at which the post was submitted

In this project we will specifically be interested in posts whose titles begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just something generally interesting.

We will compare these two types of posts to determine the following:

-  Do `Ask HN` or `Show HN` receive more comments on average?
-  Do posts created at a certain time receive more comments on average?

**Reading in the data**

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
#Extract the first row which contains the column titles
headers = hn[0]
hn = hn[1:]

In [2]:
#Let's look at the first five rows of data
print(headers)
for row in hn[:5]:
    print(row)
    

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']
['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']


**Extracting Ask HN and Show HN Posts**

Since we're only concerned with posts whose titles begin with `Ask HN` or `Show HN`, we'll create new list of lists containing just the data for those titles.

In [7]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
print("Number of Ask HN posts: ", len(ask_posts))
print("Number of Show HN posts: ", len(show_posts))
print("Number of other posts: ", len(other_posts))


Number of Ask HN posts:  1744
Number of Show HN posts:  1162
Number of other posts:  17194


**Calculating the Average Number of Comments for Ask HN and Show HN Posts**

We will now determine if ask posts or show posts receive more comments on average.

In [10]:
total_ask_comments = 0
for row in ask_posts:
    total_ask_comments += int(row[4])
avg_ask_comments = total_ask_comments/len(ask_posts)

total_show_comments = 0
for row in show_posts:
    total_show_comments += int(row[4])
avg_show_comments = total_show_comments/len(show_posts)

print("Average number of ask comments: ", avg_ask_comments)
print("Average number of show comments: ", avg_show_comments)


Average number of ask comments:  14.038417431192661
Average number of show comments:  10.31669535283993


It appears that ask posts receive more comments on average presumably because it is easier to answer a question than to provide feedback on a product or idea in a show post. 

**Finding the Amount of Ask Posts and Comments by Hour Created**

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We will use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of ask posts received by hour created.

`result_list` will be a list with two elements. The first element will be the column `created_at` and the second element will be the number of comments of the post.

In [12]:
import datetime as dt
result_list = []
for row in ask_posts:
    date_created = row[6]
    num_comments = int(row[4])
    result_list.append([date_created, num_comments])

In [17]:
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    format_string = "%m/%d/%Y %H:%M"
    datetime_object = dt.datetime.strptime(row[0], format_string)
    hour = datetime_object.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
        
    

Above, we created two dictionaries:

- `counts_by_hour`: contains the number of ask posts created during each hour of the day
- `comments_by_hour`: contains the corresponding number of comments ask posts created at each hour received

Next, we'll use these to calculate the average number of comments for posts created during each hour of the day. The result below will be a list of lists `avg_by_hour` in which the first element is the hour and the second element is the average number of comments per post

In [21]:
avg_by_hour = []
for hour in counts_by_hour:
    avg = comments_by_hour[hour]/counts_by_hour[hour]
    avg_by_hour.append([hour, avg])    

**Sorting and Displaying the Results**

We will finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [26]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

top_five = sorted_swap[:5]

print("Top 5 Hours for Ask Posts Comments")

for row in top_five:
    hour = row[1]
    avg = row[0]
    hour_dt_object = dt.datetime.strptime(hour, "%H")
    time_string = hour_dt_object.strftime("%H:%M")
    output = "{}: {:.2f} average comments per post"
    print(output.format(time_string, avg))
    

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


**Conclusions**

The hour that received the most comments per post on average is 15:00, with an average of 38.59 comments per post.

According to the data set [d